In [ ]:
import spahybgen.utils.utils_plotly as ut_plotly
from spahybgen.handmodel import HandModel
from plotly import graph_objects as go
import os
import json
import trimesh as tm
import numpy as np
import torch

os.chdir('/path to project/spahybgen')
print(os.getcwd())

### Visualize the hand contacts
The red contacts are uniformly sampled on four-vertice regions stored in handmodel/properties/contact_custom_hand.json files.

In [ ]:
robot_name = 'finray4f'
orientation_type = 'R6d'
batch_size = 1
hand_model = HandModel.load_hand_from_json(robot_name, batch_size)

init_opt_q = torch.zeros(batch_size, (3 + 6) + hand_model.actuate_dofs, device='cuda')
init_opt_q[:, :3] = torch.tensor([0.0, 0.0, 0.0], device='cuda')
init_opt_q[:, 3:9] = torch.tensor([1., 0., 0., 0., 1., 0.], device='cuda')
init_opt_q[:, 9:] = (hand_model.joints_q_lower + hand_model.joints_q_upper)/2

vis_data = hand_model.get_plotly_data(init_opt_q, color='lightblue', opacity=0.6)

contact_points, contact_normals = hand_model.get_sampled_contact_points_and_normal(q=init_opt_q)
vis_data.append(ut_plotly.plot_point_cloud(pts=contact_points.cpu().squeeze(0), color='red'))
fig = go.Figure(data=vis_data)
fig.show()


## Choose four vertices of targeted contact regions 
The following code visualize all stl parts of the robotic hand.
Hovering the cursor to the vertices of targeted contact regions to inspect the labels, and add the number labels of four vertices (in counter-clockwise sequence) for every desired contact regions to the contact_custom_hand.json file.

In [ ]:
robot_name = 'finray4f'
base_path = "/path to project/spahybgen/" + "handmodel/urdf/" + robot_name + "/meshes/"

mesh_i = 0
for mesh_i in range(len(hand_model.robot_full.links)):
    if len(hand_model.robot_full.links[mesh_i].visuals) == 0: continue
    mesh_path = hand_model.robot_full.links[mesh_i].visuals[0].geometry.filename
    link_name = hand_model.robot_full.links[mesh_i].name
    if link_name[:6] != 'thumb_': continue 
    print('"' + link_name + '"', ': [, , , ],')

    mesh = tm.load(base_path + mesh_path, force='mesh', process=False)
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)
    vis_data = []
    vis_data.append(go.Mesh3d(x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2], 
    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2], color="lightblue", opacity=1.0, hovertext=[str(hi) for hi in range(vertices.shape[0])]))
    # indxs = np.array([1823, 1809, 1804, 1828])
    # vis_data.append(ut_plotly.plot_point_cloud(pts=vertices[indxs, :], color='red'))
    fig = go.Figure(data=vis_data).show()